# Stereometry Notebook

## Preliminaries

In [9]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import os

## Processing raw data

### Declaring functions

In [10]:
# ======================================================
# 1. Detect green dots in one image
# ======================================================
def detect_green_dots(image_path):
    """
    Detect neon-green dots in an image using HSV thresholding + connected components.
    Returns a list of (x, y) centroids.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define HSV threshold range for neon green points
    lower_green = np.array([40, 100, 100])
    upper_green = np.array([80, 255, 255])

    # Apply threshold
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Morphological cleaning
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Connected components
    num, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)

    points = []
    for i in range(1, num):  # skip background
        cx, cy = centroids[i]
        points.append((int(cx), int(cy)))

    return points


# ======================================================
# 2. Process images (single file or folder)
# ======================================================
def process_images(input_path, output_folder="detections"):
    """
    Process either a single image or all images in a folder.
    Each image produces its own CSV with detected green dots.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Case 1: single image
    if os.path.isfile(input_path):
        files = [input_path]

    # Case 2: folder of images
    elif os.path.isdir(input_path):
        files = [
            os.path.join(input_path, f)
            for f in os.listdir(input_path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp'))
        ]
    else:
        raise ValueError(f"Path not found: {input_path}")

    for f in files:
        try:
            detections = detect_green_dots(f)
        except Exception as e:
            print(f"❌ Error processing {f}: {e}")
            continue

        out_name = os.path.splitext(os.path.basename(f))[0] + ".csv"
        out_path = os.path.join(output_folder, out_name)

        df = pd.DataFrame(detections, columns=["x", "y"])
        df.to_csv(out_path, index=False)

        print(f"✅ Processed {f}, found {len(df)} dots → saved {out_path}")


### Calling functions

In [11]:
for i in range(3, 10):
    process_images(f"Pictures/{i}.png")

✅ Processed Pictures/3.png, found 33 dots → saved detections\3.csv
✅ Processed Pictures/4.png, found 34 dots → saved detections\4.csv
✅ Processed Pictures/5.png, found 33 dots → saved detections\5.csv
✅ Processed Pictures/6.png, found 17 dots → saved detections\6.csv
✅ Processed Pictures/7.png, found 0 dots → saved detections\7.csv
✅ Processed Pictures/8.png, found 0 dots → saved detections\8.csv
✅ Processed Pictures/9.png, found 0 dots → saved detections\9.csv


## Solving for Z components
We are given the formula:
$$ Z = \frac{bf}{x_1 - x_2}. $$

In [4]:
# Define function from equation above
def z_solver(x1, x2, b, f):
    return (b*f)/(x1-x2)

## Coordinates of Sample Points

In [ ]:
# Ingest with pd.read_csv()